In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt
sns.set(style="whitegrid")

# TEXT WRAP
import textwrap
import pprint
import itertools

# STAT MODELS
from scipy import stats
import statsmodels.api as sm
from sklearn import linear_model as lm
from sklearn.metrics import r2_score

# IMPORT UTILS PY FILE
from google.colab import files
import sys 
sys.path.append('/content/drive/My Drive/python-code')
# open('python_data_utility.py','wb').write(src)

PYCODE_PATH = "/content/drive/My Drive/python-code" # This is a custom path.
# PYCODE_PATH = join(ROOT, PROJ)

from importlib.machinery import SourceFileLoader
ut = SourceFileLoader('python_data_utility', PYCODE_PATH + '/python_data_utility.py').load_module()
# import python_data_utility as ut

from IPython.display import display, HTML, IFrame

#directories
wd = "/content/drive/My Drive/PhD Datasets/Lending-club/"
mainpath = '/content/drive/My Drive/'

%matplotlib inline
# activate R magic
%load_ext rpy2.ipython

# ut.wprint('Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.', 80)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [0]:
src = list(files.upload().values())[0]

# Load data

In [0]:
def wprint(str, N=80):
  for i, col in enumerate(str,1):
    if (i % N) == 0:
      print(str[i-N:i])
    else: continue

In [0]:
funda = pd.read_stata(mainpath + 'Andre - PhD data and codes/data/funda_v3.dta')
compu = pd.read_stata(mainpath + 'Andre - PhD data and codes/rawdata/Compustat/compqt19782016.dta')

In [0]:
# compu.describe()
funda.shape

(328909, 627)

In [0]:
# display list of column
sep = ' '
varlist = sep.join(funda.columns)

wprint(varlist, 90)

gvkey gvkey_num datadate fyear indfmt consol popsrc datafmt tic cusip conm curcd fyr scf c
ik costat fic busdesc fyrc idbflag loc sic ipodate acqintan act am aoloch ap apalch aqc aq
p at bkvlps capx cdvc ceq ceqt ch che chech cogs csho dd1 dd2 dd3 dd4 dd5 dfxa dlc dlcch d
ltis dltr dltt dp dpc ds dt dv dvc dvp dvpa dvpd dxd2 dxd3 dxd4 dxd5 ebit emol emp esub es
ubc exre fiao fincf fopo fsrco fuseo gdwl ibc idit intan intano invch invt ivaco ivch ivnc
f ivst ivstch lcal lco lct lo lt mii ni np oancf oiadp pdvc pi pidom ppegt ppent prstkc re
 recch rect sale siv sppe sppiv sstk stbo tlcf txach txdb txdc txdi txfed txfo txo txp txp
d txs txt udpfa ustdnc utfdoc uxinst uxintd wcap wcapc wcapch xad xidoc xint xintd xrd xrd
p xrent mkvalt prcc_f intpn dcs niint niit dvpsp_f xsga sic3dg sic2dg permno permco cusip6
 cusip8 tpci age maxage cpi2005 tvar atgrw salegrw lnsalegrw mkvalt1 tobQ exclu_sic exclu_
firm exclu_age exclu_fd exclu_all Invest CF1 CF2 ChgDebt ChgEqui na m_na C_CheLM lnat Tang

In [0]:
# run regression
xVarList = ['refinTD1', 'lnat', 'tobQ', 'std_cfo', 'std_sale', 'AltmanZ', 'K', 'Ind_K', 'Slack', 'CFOSale']
yVarList = ['OverInvest1']
 
# >>> drop na
funda_noNaN = funda.dropna(axis=0,how="any",subset=(xVarList+yVarList))
funda_noNaN.shape

X = funda_noNaN[xVarList] #predictor
Y = funda_noNaN[yVarList] #response

# Regressions

##Linear regression in Python using Sklearn

In [0]:
# 1st APPROACH USE STATMODEL (simple) =============

# alt 1 ===================
# model = sm.OLS(Y, X)
# results = model.fit()
# print(results.summary())

In [0]:
# 2nd APPROACH - SKLEARN PACKAGE ===================
regr = lm.LinearRegression()
regr.fit(X, Y)
# # >>>coef_ : array, shape (n_features, ) or (n_targets, n_features)
# print(regr.coef_)
# # >>>intercept_ : array  Independent term in the linear model.
# print(regr.intercept_)

# print R^2 for the training set
print('The R-squared value for the training set is: {:0.4f}'.format(r2_score(Y, regr.predict(X))))

# print intercept and coefficients
param_df = pd.DataFrame({"Coefficient": [regr.intercept_] + list(regr.coef_[0]),
"Feature": ['intercept'] + list(xVarList)})

param_df[['Feature', 'Coefficient']]


---




# Linear regression using R engine

In [0]:
#save df1 to local drive
funda_noNaN.to_csv("/content/drive/My Drive/Andre - PhD data and codes/data/funda_v3.csv",index=False)

In [0]:
%%R
# install R packages
# install.packages('knitr')
# install.packages('data.table')
# install.packages('outreg')
# install.packages('lubridate')
# install.packages('stargazer')
install.packages('xtable')
install.packages('ddpcr')
library(lubridate)
library(outreg)
library(foreign)
library(tidyverse)
library(stargazer)
library(ddpcr)
library(xtable)
library(IRdisplay)
library(repr)
# The downloaded source packages are in 	‘/tmp/RtmpPQDaPt/downloaded_packages’

In [0]:
# load data to R dataset
%%R
file = "/content/drive/My Drive/Andre - PhD data and codes/data/funda_v3.csv"
dataset =  read.csv(file)
rdf <- data.frame(dataset)
#show columns name
names(rdf)

In [0]:
%%R
# Single regression
# for (val in xVarlists) {
# ExVar[i] <- toString(paste(xVarList, "+ ", collapse = ''))
# ExVar <- substr(ExVar, 1, nchar(ExVar)-3) #<-- this to remove the last '+'
# }
ctrlvarlist = c('lnat', 'tobQ', 'std_cfo', 'std_sale', 'AltmanZ', 'K', 'Ind_K', 'Slack', 'CFOSale')
xVarList = c('refinTD1', ctrlvarlist)

ExVar <- toString(paste(xVarList, "+ ", collapse = ''))
ExVar <- substr(ExVar, 1, nchar(ExVar)-3) #<-- this to remove the last '+'
model1 <- paste("OverInvest1 ~ ",ExVar) 
print(model1)

fit1 <- lm(eval(parse(text = model1)),data = rdf)
summary(fit1)

In [0]:
%%R
ctrlvarlist = c('lnat', 'tobQ', 'std_cfo', 'std_sale', 'AltmanZ', 'K', 'Ind_K', 'Slack', 'CFOSale')
xVarList1 = c('refinTD1', ctrlvarlist)
xVarList2 = c('refinTD2', ctrlvarlist)
xVarList3 = c('refinLT', ctrlvarlist)
xVarlists <- vector(mode = "list", 3) #initialize empty list for xVarlists
ExVars <- vector(mode = "list", 3) #initialize empty list for ExVars
Models <- vector(mode = "list", 3) #Models hold list of model
Fits <- vector(mode = "list", 3) #ModelFits hold regression objs

xVarlists = list(toString(xVarList1),toString(xVarList2), toString(xVarList3))
for (i in seq_along(xVarlists)){
    xVarlists[i] = str_replace(xVarlists[i], ', ', ' + ')
}

for (i in seq_along(xVarlists)) {
ExVars[i] <- toString(paste(xVarlists[i], "+ ", collapse = ''))
ExVars[i] <- str_replace_all(substr(ExVars[i], 1, nchar(ExVars[i])-3),', ',' + ') #<-- this to remove the last '+'
Models[i] <- paste("OverInvest1 ~ ",ExVars[i])
Fits[i] <- lm(eval(parse(text = Models[i])),data = rdf)
}
# Output regression
# ================================================
# USING OUTREG PACKAGES
# fitlist <- list(lm(eval(parse(text = Models[1])),data = rdf),
#                 lm(eval(parse(text = Models[2])),data = rdf),
#                 lm(eval(parse(text = Models[3])),data = rdf)
#                 )
# outreg(fitlist)
# ===============================================
# ================================================
# USING STARGAZER PACKAGES

RESULTFOLDER = '/content/drive/My Drive/Andre - PhD data and codes/results/'
resultfile = paste(RESULTFOLDER, "refin-over-models.htm", sep = '', collapse = '')
print(resultfile)

quiet(
stargazer(
    lm(eval(parse(text = Models[1])),data = rdf),
    lm(eval(parse(text = Models[2])),data = rdf),
    lm(eval(parse(text = Models[3])),data = rdf),
    type="html",
    dep.var.labels=c("Overinvestment"),
#  covariate.labels=c("Gross horsepower","Rear axle ratio","Four foward gears",
#  "Five forward gears","Type of transmission (manual=1)"), 
    out=resultfile)
)
# =================================================

[1] "/content/drive/My Drive/Andre - PhD data and codes/results/refin-over-models.htm"


In [0]:
RESULTFOLDER = '/content/drive/My Drive/Andre - PhD data and codes/results/'
resultfile = RESULTFOLDER + "refin-over-models.htm"

display(HTML(filename=resultfile))

In [0]:
mydata$fast <- as.numeric((mydata$mpg > 20.1)) #Creating a dummy variable 1 = fast car
m1 <- lm(mpg ~ hp, data=mydata)
m2 <- lm(mpg ~ hp + drat, data=mydata)
m3 <- lm(mpg ~ hp + drat + factor(gear), data=mydata)
m4 <- glm(fast ~ hp + drat + am, family=binomial(link="logit"), data=mydata)
stargazer(m1, m2, m3, m4, type="text",
 dep.var.labels=c("Miles/(US) gallon","Fast car (=1)"),
 covariate.labels=c("Gross horsepower","Rear axle ratio","Four foward gears",
 "Five forward gears","Type of transmission (manual=1)"), out="models.txt")